In [ ]:
%config IPCompleter.use_jedi = False
%pdb off
# %load_ext viztracer
# from viztracer import VizTracer
%load_ext autoreload
%autoreload 3
import sys
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Callable, Union, Any
from typing_extensions import TypeAlias
from nptyping import NDArray
# import neuropy.utils.type_aliases as types
import numpy as np
import pandas as pd


# required to enable non-blocking interaction:
%gui qt5

## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
# from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
# from pyphoplacecellanalysis.General.Mixins.ExportHelpers import session_context_to_relative_path
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.utils.result_context import IdentifyingContext


from pyphoplacecellanalysis.GUI.Qt.Widgets.DebugWidgetStylesheetInspector import ConnectStyleSheetInspector
from qtpy import QtCore, QtWidgets, QtGui
from pyphocorehelpers.gui.Qt.ExceptionPrintingSlot import pyqtExceptionPrintingSlot
from typing import Dict, List, Tuple, Optional, Callable, Union, Any
import pyphoplacecellanalysis.External.pyqtgraph as pg

from pyphocorehelpers.DataStructure.general_parameter_containers import VisualizationParameters
from pyphocorehelpers.gui.Qt.GlobalConnectionManager import GlobalConnectionManager, GlobalConnectionManagerAccessingMixin
from pyphocorehelpers.gui.Qt.qevent_lookup_helpers import QEventLookupHelpers
from pyphocorehelpers.programming_helpers import metadata_attributes
from pyphocorehelpers.function_helpers import function_attributes

from PyQt5.QtCore import Qt
from PyQt5.QtGui import QKeySequence

# from pyphoplacecellanalysis.General.Batch.runBatch import BatchRun, BatchResultDataframeAccessor, run_diba_batch, SessionBatchProgress, main
from neuropy.utils.result_context import IdentifyingContext
from pyphocorehelpers.assertion_helpers import Assert
from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import loadData
from pyphocorehelpers.Filesystem.path_helpers import set_posix_windows
from pyphoplacecellanalysis.General.Batch.pythonScriptTemplating import generate_batch_single_session_scripts


from pyphoplacecellanalysis.GUI.Qt.SpikeRasterWindows.Spike3DRasterWindowWidget import Spike3DRasterWindowWidget
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import _setup_spike_raster_window_for_debugging

global_data_root_parent_path = find_first_extant_path([Path(r'W:\Data'), Path(r'/home/halechr/FastData'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data')])
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"

# ==================================================================================================================== #
# Load Data                                                                                                            #
# ==================================================================================================================== #

active_data_mode_name = 'kdiba'
local_session_root_parent_context = IdentifyingContext(format_name=active_data_mode_name) # , animal_name='', configuration_name='one', session_name=a_sess.session_name
local_session_root_parent_path = global_data_root_parent_path.joinpath('KDIBA')

# Hardcoded included_session_contexts:
included_session_contexts = [
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54'), # prev completed
]
## Get the context:basedir dict for the relevant contexts:
output_session_basedir_dict = {k:v for k, v in KDibaOldDataSessionFormatRegisteredClass.build_session_basedirs_dict(global_data_root_parent_path).items() if k in included_session_contexts}
# output_session_basedir_dict


In [ ]:
included_session_contexts, output_python_scripts, output_slurm_scripts = generate_batch_single_session_scripts(global_data_root_parent_path, session_batch_basedirs=output_session_basedir_dict, included_session_contexts=included_session_contexts, output_directory=Path('../output/gen_scripts/').resolve(),  use_separate_run_directories=True, create_slurm_scripts=False,
	should_force_reload_all=False, should_perform_figure_generation_to_file=False)
output_python_scripts

In [ ]:
%pdb off
# def _subfn_run_main(pkl_path, debug_print:bool=True, debug_mode: bool=True):
# 	""" run main function to perform batch processing. """
# 	print(f'pkl_path: {pkl_path}')
	
# 	def _try_load_global_batch_result():
# 		if debug_print:
# 			print(f'pkl_path: {pkl_path}')
# 		# try to load an existing batch result:
# 		try:
# 			curr_active_pipeline = loadData(pkl_path, debug_print=debug_print)
			
# 		except NotImplementedError:
# 			# Fixes issue with pickled POSIX_PATH on windows for path.                    
# 			with set_posix_windows():
# 				curr_active_pipeline = loadData(pkl_path, debug_print=debug_print) # Fails this time if it still throws an error

# 		except (FileNotFoundError, TypeError):
# 			# loading failed
# 			print(f'Failure loading {pkl_path}.')
# 			curr_active_pipeline = None
			
# 		return curr_active_pipeline
	
# 	curr_active_pipeline = _try_load_global_batch_result()
# 	assert curr_active_pipeline is not None
# 	print(f'loaded `curr_active_pipeline`, building Spike3DRasterWindowWidget...')
	
# 	## Loads `curr_active_pipeline` from the provided path
# 	if debug_mode:
# 		Spike3DRasterWindowWidget.enable_interaction_events_debug_print = True    
# 	# Gets the existing SpikeRasterWindow or creates a new one if one doesn't already exist:
# 	spike_raster_window, (active_2d_plot, active_3d_plot, main_graphics_layout_widget, main_plot_widget, background_static_scroll_plot_widget) = Spike3DRasterWindowWidget.find_or_create_if_needed(curr_active_pipeline, force_create_new=True)
# 	print(f'built `Spike3DRasterWindowWidget`, launching...')
# 	if debug_mode:
# 		spike_raster_window.enable_debug_print = True
# 		Spike3DRasterWindowWidget.enable_interaction_events_debug_print = True
# 		print(f'\tspike_raster_window.should_debug_print_interaction_events: {spike_raster_window.should_debug_print_interaction_events}')
	
# 	all_global_menus_actionsDict, global_flat_action_dict = _setup_spike_raster_window_for_debugging(spike_raster_window, debug_print=debug_print)

# 	return spike_raster_window, (active_2d_plot, active_3d_plot, main_graphics_layout_widget, main_plot_widget, background_static_scroll_plot_widget), curr_active_pipeline


debug_print:bool=True
debug_mode: bool=True

""" Usage:

python scripts/runSingleBatch.py --active_result_suffix "2023-08-08_bApogee" --num_processes 4 --should_force_reload_all --debug_print
python scripts/runSingleBatch.py --active_result_suffix "2023-08-08_LNX00052" --num_processes 4 --should_force_reload_all --debug_print

--should_perform_figure_generation_to_file
--should_perform_figure_generation_to_file

"""
# parser = argparse.ArgumentParser(description='Perform batch processing.')
# parser.add_argument('--active_result_suffix', required=True, help='Suffix used for filenames throughout the notebook.')
# parser.add_argument('--num_processes', type=int, default=1, help='Number of processes to use.')
# parser.add_argument('--should_force_reload_all', action='store_true', help='Force reload all data.')
# parser.add_argument('--should_perform_figure_generation_to_file', action='store_true', help='Perform figure generation to file.')
# parser.add_argument('--debug_print', action='store_true', help='Enable debug printing.')

# args = parser.parse_args()

pkl_path = Path('W:/Data/KDIBA/gor01/one/2006-6-09_1-22-43/loadedSessPickle.pkl')
# pkl_path = Path('W:/Data/KDIBA/vvp01/one/2006-4-09_17-29-30/loadedSessPickle.pkl')
# pkl_path = Path('W:/Data/KDIBA/vvp01/one/2006-4-09_17-29-30/loadedSessPickle.pkl')
# pkl_path = Path('W:/Data/KDIBA/vvp01/two/2006-4-10_12-58-3/loadedSessPickle.pkl')
Assert.path_exists(pkl_path)

# app = QtWidgets.QApplication(sys.argv)
app = pg.mkQApp('2025-01-07 - Spike3DRasterWindowWidget testing notebook')
# Define stylesheet
app.setStyleSheet("""
	VContainer {
		background-color: #00e1ff; /* Light gray background */
		color: #ff0095;           /* Text color */
		border: 1px solid #4400ff; /* Border around the container */
	}
	VContainer QLabel {
		color: #ffd000; /* Red text inside VContainer's labels */
	}
	QSplitter::handle { height: 2px; width: 2px; }
""")

# testWidget = Spike3DRasterWindowWidget()



# spike_raster_window, (active_2d_plot, active_3d_plot, main_graphics_layout_widget, main_plot_widget, background_static_scroll_plot_widget), curr_active_pipeline = _subfn_run_main(pkl_path)

print(f'pkl_path: {pkl_path}')
	
def _try_load_global_batch_result():
	if debug_print:
		print(f'pkl_path: {pkl_path}')
	# try to load an existing batch result:
	try:
		curr_active_pipeline = loadData(pkl_path, debug_print=debug_print)
		
	except NotImplementedError:
		# Fixes issue with pickled POSIX_PATH on windows for path.                    
		with set_posix_windows():
			curr_active_pipeline = loadData(pkl_path, debug_print=debug_print) # Fails this time if it still throws an error

	except (FileNotFoundError, TypeError):
		# loading failed
		print(f'Failure loading {pkl_path}.')
		curr_active_pipeline = None
		
	return curr_active_pipeline

curr_active_pipeline = _try_load_global_batch_result()
assert curr_active_pipeline is not None
print(f'loaded `curr_active_pipeline`, building Spike3DRasterWindowWidget...')


In [ ]:
enable_interaction_events_debug_print = False
## Loads `curr_active_pipeline` from the provided path
if debug_mode:
	# Spike3DRasterWindowWidget.enable_interaction_events_debug_print = True
	Spike3DRasterWindowWidget.enable_interaction_events_debug_print = enable_interaction_events_debug_print   
# Gets the existing SpikeRasterWindow or creates a new one if one doesn't already exist:
spike_raster_window, (active_2d_plot, active_3d_plot, main_graphics_layout_widget, main_plot_widget, background_static_scroll_plot_widget) = Spike3DRasterWindowWidget.find_or_create_if_needed(curr_active_pipeline, force_create_new=True)


In [ ]:
print(f'built `Spike3DRasterWindowWidget`, launching...')
if debug_mode:
	spike_raster_window.enable_debug_print = True
	# Spike3DRasterWindowWidget.enable_interaction_events_debug_print = enable_interaction_events_debug_print
	Spike3DRasterWindowWidget.enable_interaction_events_debug_print = True
	print(f'\tspike_raster_window.should_debug_print_interaction_events: {spike_raster_window.should_debug_print_interaction_events}')

all_global_menus_actionsDict, global_flat_action_dict = _setup_spike_raster_window_for_debugging(spike_raster_window, debug_print=debug_print)

# spike_raster_window.show()

# testWidget.show()

print(f'waiting...')
# ConnectStyleSheetInspector(main_window=spike_raster_window, shortcut=QKeySequence(Qt.CTRL + Qt.SHIFT + Qt.Key_F12)) # Connects the global stylesheet inspector to the window


# sys.exit(app.exec_())

# Accessing Properties of `active_2d_plot`:

In [6]:
# preview_overview_scatter_plot: pg.ScatterPlotItem  = active_2d_plot.plots.preview_overview_scatter_plot # ScatterPlotItem 
# preview_overview_scatter_plot.setDownsampling(auto=True, method='subsample', dsRate=10)
main_graphics_layout_widget: pg.GraphicsLayoutWidget = active_2d_plot.ui.main_graphics_layout_widget
wrapper_layout: pg.QtWidgets.QVBoxLayout = active_2d_plot.ui.wrapper_layout
main_content_splitter = active_2d_plot.ui.main_content_splitter # QSplitter
layout = active_2d_plot.ui.layout
background_static_scroll_window_plot = active_2d_plot.plots.background_static_scroll_window_plot # PlotItem
main_plot_widget = active_2d_plot.plots.main_plot_widget # PlotItem

# Set stretch factors to control priority
main_graphics_layout_widget.ci.layout.setRowStretchFactor(0, 1)  # Plot1: lowest priority
main_graphics_layout_widget.ci.layout.setRowStretchFactor(1, 2)  # Plot2: medium priority

main_plot_widget.setMinimumHeight(0.0)
# Hide the first plot
main_plot_widget.hide()  # This will make plot1 invisible but still part of the layout

# main_plot_widget.setFixedHeight(20.0)
# main_plot_widget.setHeight(20.0)
# main_plot_widget.setMinimumHeight(0.0)
background_static_scroll_window_plot.setMinimumHeight(50.0)
background_static_scroll_window_plot.setMaximumHeight(75.0)
# background_static_scroll_window_plot.setFixedHeight(50.0)


active_2d_plot.params.use_docked_pyqtgraph_plots = True
# Set stretch factors to control priority
main_graphics_layout_widget.ci.layout.setRowStretchFactor(0, 1)  # Plot1: lowest priority
main_graphics_layout_widget.ci.layout.setRowStretchFactor(1, 2)  # Plot2: mid priority
main_graphics_layout_widget.ci.layout.setRowStretchFactor(2, 3)  # Plot3: highest priority


In [ ]:
active_2d_plot.plots_data

# Includes2DActiveWindowScatter
self.plots.scatter_plot
self.plots_data.all_spots


In [ ]:
background_static_scroll_window_plot # PlotItem
background_static_scroll_window_plot.getViewWidget()
background_static_scroll_window_plot.allChildItems()

# [<pyphoplacecellanalysis.External.pyqtgraph.graphicsItems.ViewBox.ViewBox.ViewBox at 0x23b7db21550>,
#  <PyQt5.QtWidgets.QGraphicsRectItem at 0x23b7db21700>,
#  <pyphoplacecellanalysis.External.pyqtgraph.graphicsItems.ViewBox.ViewBox.ChildGroup at 0x23b7db21670>,
#  <pyphoplacecellanalysis.External.pyqtgraph.graphicsItems.ScatterPlotItem.ScatterPlotItem at 0x23b7ec6e820>,
#  <pyphoplacecellanalysis.External.pyqtgraph.graphicsItems.InfiniteLine.InfiniteLine at 0x23b78534a60>,
#  <pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.GraphicsObjects.IntervalRectsItem.IntervalRectsItem at 0x23b9c4de3a0>,
#  <pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.GraphicsObjects.IntervalRectsItem.IntervalRectsItem at 0x23b9c4de310>,
#  <pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.GraphicsObjects.IntervalRectsItem.IntervalRectsItem at 0x23b9c4de820>,
#  <pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.GraphicsObjects.CustomLinearRegionItem.CustomLinearRegionItem at 0x23a9ecf3c10>,
#  <pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.GraphicsObjects.CustomInfiniteLine.CustomInfiniteLine at 0x23b1ff8fb80>,
#  <pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.GraphicsObjects.CustomInfiniteLine.CustomInfiniteLine at 0x23b1ff8ff70>,
#  <PyQt5.QtWidgets.QGraphicsRectItem at 0x23b7db21820>,
#  <PyQt5.QtWidgets.QGraphicsRectItem at 0x23b7db218b0>,
#  <PyQt5.QtWidgets.QGraphicsRectItem at 0x23b7db21790>,
#  <pyphoplacecellanalysis.External.pyqtgraph.graphicsItems.ButtonItem.ButtonItem at 0x23b7db21430>,
#  <pyphoplacecellanalysis.External.pyqtgraph.graphicsItems.LabelItem.LabelItem at 0x2397d3483a0>,
#  <PyQt5.QtWidgets.QGraphicsTextItem at 0x2397d348430>,
#  <pyphoplacecellanalysis.External.pyqtgraph.graphicsItems.AxisItem.AxisItem at 0x2397d31fee0>,
#  <PyQt5.QtWidgets.QGraphicsTextItem at 0x2397d31ff70>,
#  <pyphoplacecellanalysis.External.pyqtgraph.graphicsItems.AxisItem.AxisItem at 0x2397d348040>,
#  <PyQt5.QtWidgets.QGraphicsTextItem at 0x2397d3480d0>,
#  <pyphoplacecellanalysis.External.pyqtgraph.graphicsItems.AxisItem.AxisItem at 0x2397d348160>,
#  <PyQt5.QtWidgets.QGraphicsTextItem at 0x2397d3481f0>,
#  <pyphoplacecellanalysis.External.pyqtgraph.graphicsItems.AxisItem.AxisItem at 0x2397d348280>,
#  <PyQt5.QtWidgets.QGraphicsTextItem at 0x2397d348310>]


In [ ]:
background_static_scroll_window_plot

In [ ]:
from copy import deepcopy


duplicate_background_static_scroll_window_plot = background_static_scroll_window_plot
# duplicate_background_static_scroll_window_plot = deepcopy(background_static_scroll_window_plot) # TypeError: cannot pickle 'PlotItem' object
duplicate_background_static_scroll_window_plot

In [ ]:
# background_static_scroll_window_plot.vb
background_static_scroll_window_plot.getViewBox()
main_plot_widget.getViewBox()

In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.DockingWidgets.DynamicDockDisplayAreaContent import CustomDockDisplayConfig, CustomCyclicColorsDockDisplayConfig, NamedColorScheme
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.Render2DScrollWindowPlot import Render2DScrollWindowPlotMixin

dock_config = CustomCyclicColorsDockDisplayConfig(named_color_scheme=NamedColorScheme.grey, showCloseButton=True, corner_radius=0)
name = f'background_static_scroll_timeline'
background_static_scroll_time_sync_pyqtgraph_widget, background_static_scroll_root_graphics_layout_widget, background_static_scroll_plot_item = active_2d_plot.add_new_embedded_pyqtgraph_render_plot_widget(name=name, dockSize=(500, 120), display_config=dock_config)
# _interval_tracks_out_dict[name] = (dock_config, background_static_scroll_time_sync_pyqtgraph_widget, background_static_scroll_root_graphics_layout_widget, background_static_scroll_plot_item)

In [23]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.Render2DScrollWindowPlot import Render2DScrollWindowPlotMixin
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import new_plot_raster_plot, NewSimpleRaster
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.GraphicsObjects.CustomInfiniteLine import CustomInfiniteLine
import pyphoplacecellanalysis.External.pyqtgraph as pg


def plot_test_spike_scatter_figure(self, defer_show = False):
	""" plots a raster plot showing the first spike for each PBE for each cell (rows) relative to the first lap spike (t=0)
	
	test_obj: CellsFirstSpikeTimes = CellsFirstSpikeTimes.init_from_batch_hdf5_exports(first_spike_activity_data_h5_files=first_spike_activity_data_h5_files)
	app, win, plots, plots_data = test_obj.plot_first_lap_spike_relative_first_PBE_spike_scatter_figure()
	
	"""
	from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import new_plot_raster_plot, NewSimpleRaster
	from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.GraphicsObjects.CustomInfiniteLine import CustomInfiniteLine
	import pyphoplacecellanalysis.External.pyqtgraph as pg

	## INPUTS: active_config
	# type(active_config.plotting_config.pf_colormap)
	## align to first lap spike (first_spike_lap)
	self.all_cells_first_spike_time_df['lap_spike_relative_first_spike'] = self.all_cells_first_spike_time_df['first_spike_PBE'] - self.all_cells_first_spike_time_df['first_spike_lap']
	# self.all_cells_first_spike_time_df['color'] = self.all_cells_first_spike_time_df['aclu'].map(lambda x: aclu_to_color_map.get(x, [1.0, 1.0, 0.0, 1.0]))
	# column_names = ['first_spike_any', 'first_spike_theta', 'first_spike_lap', 'first_spike_PBE']

	## plot the spike timecourse:
	# spike_scatter_kwargs = dict(s=25)

	## find extrema
	# active_col_names = column_names
	active_col_names = ['lap_spike_relative_first_spike', ]
	earliest_first_spike_t: float = self.all_cells_first_spike_time_df[active_col_names].min(axis=0).min()
	latest_first_spike_t: float = self.all_cells_first_spike_time_df[active_col_names].max(axis=0).max()
	# ax.set_xlim(earliest_first_spike_t, latest_first_spike_t)


	# _temp_active_spikes_df = deepcopy(test_obj.all_cells_first_spike_time_df)[['aclu', 'neuron_uid', 'lap_spike_relative_first_spike']].rename(columns={'lap_spike_relative_first_spike':'t_rel_seconds'})
	_temp_active_spikes_df = deepcopy(self.all_cells_first_spike_time_df)[['neuron_uid', 'lap_spike_relative_first_spike']].rename(columns={'lap_spike_relative_first_spike':'t_rel_seconds'})
	# Use pd.factorize to create new integer codes for 'neuron_uid'
	_temp_active_spikes_df['aclu'], uniques = pd.factorize(_temp_active_spikes_df['neuron_uid'])
	# Optionally, add 1 to start 'aclu' from 1 instead of 0
	_temp_active_spikes_df['aclu'] = _temp_active_spikes_df['aclu'] + 1
	# Now, 'aclu' contains unique integer IDs corresponding to 'neuron_uid'
	print(_temp_active_spikes_df[['neuron_uid', 'aclu']].drop_duplicates())

	_temp_active_spikes_df
	# shared_aclus = deepcopy(_temp_active_spikes_df['neuron_uid'].unique())
	shared_aclus = deepcopy(_temp_active_spikes_df['aclu'].unique())
	shared_aclus
	# Assuming _temp_active_spikes_df is your DataFrame


	app, win, plots, plots_data = new_plot_raster_plot(_temp_active_spikes_df, shared_aclus, scatter_plot_kwargs=None,
														scatter_app_name=f'lap_spike_relative_first_spike_raster', defer_show=defer_show, active_context=None)

	root_plot = plots['root_plot']
	# Create a vertical line at x=3
	v_line = CustomInfiniteLine(pos=0.0, angle=90, pen=pg.mkPen('r', width=2), label='first lap spike')
	root_plot.addItem(v_line)
	plots['v_line'] = v_line
	
	## Set Labels
	# plots['root_plot'].set_xlabel('First PBE spike relative to first lap spike (t=0)')
	# plots['root_plot'].set_ylabel('Cell')
	plots['root_plot'].setTitle("First PBE spike relative to first lap spike (t=0)", color='white', size='24pt')
	# plots['root_plot'].setLabel('top', 'First PBE spike relative to first lap spike (t=0)', size='22pt') # , color='blue'
	plots['root_plot'].setLabel('left', 'Cell ID', color='white', size='12pt') # , units='V', color='red'
	plots['root_plot'].setLabel('bottom', 'Time (relative to first lap spike for each cell)', color='white', units='s', size='12pt') # , color='blue'


	return app, win, plots, plots_data


app, win, plots, plots_data = new_plot_raster_plot(_temp_active_spikes_df, shared_aclus)



In [ ]:
## INPUT: background_static_scroll_window_plot
background_static_scroll_plot_item.vb # CustomViewBox 
background_static_scroll_plot_item.getViewBox() # CustomViewBox 

# main_plot_widget = self.plots.main_plot_widget # PlotItem
background_static_scroll_plot_item.setXLink(main_plot_widget) # works to synchronize the main zoomed plot (current window) with the epoch_rect_separate_plot (rectangles plotter)


In [ ]:
background_static_scroll_window_plot

background_static_scroll_plot_item.addItem

In [ ]:
long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
global_spikes_df = deepcopy(curr_active_pipeline.computation_results[global_epoch_name]['computed_data'].pf1D.spikes_df)
_out_ripple_rasters: RankOrderRastersDebugger = RankOrderRastersDebugger.init_rank_order_debugger(global_spikes_df, deepcopy(filtered_ripple_simple_pf_pearson_merged_df),
												track_templates, None,
													None, None,
													dock_add_locations = dict(zip(('long_LR', 'long_RL', 'short_LR', 'short_RL'), (['right'], ['right'], ['right'], ['right']))),
													)

# List Selector:

In [ ]:
from pyphocorehelpers.gui.Jupyter.simple_widgets import build_dropdown_selection_widget

active_slected_context = included_session_contexts[0]

def selected_context_changed_callback(context_str, context_data):
    global active_slected_context
    print(f"You selected {context_str}:{context_data}")
    active_slected_context = context_data # update the selected context

context_selection_dropdown = build_dropdown_selection_widget(included_session_contexts, on_user_update_item_selection=selected_context_changed_callback)
context_selection_dropdown

In [ ]:
active_slected_context

In [ ]:

def selected_context_changed_callback(change):
    selected_item = change['new']
    print(f"You selected {selected_item}")
    # Run your function or cell code here

# item_list = ['Item 1', 'Item 2', 'Item 3']
item_list = included_session_contexts.copy()
context_selection_dropdown = widgets.Dropdown(options=item_list, description='Select Context:', layout=widgets.Layout(width='auto'))
context_selection_dropdown.style = {'description_width': 'initial'} # Increase dropdown width with CSS
context_selection_dropdown.observe(selected_context_changed_callback, 'value')
display(context_selection_dropdown)


# Tree selector:

In [ ]:
from pyphocorehelpers.gui.Jupyter.TreeWidget import JupyterTreeWidget

jupyter_tree_widget = JupyterTreeWidget(included_session_contexts=included_session_contexts)
# jupyter_tree_widget

In [ ]:
from collections import defaultdict

def construct_tree(lst, keys):
    tree = defaultdict(list)
    if len(keys) == 1:
        for d in lst:
            tree[d[keys[0]]].append(d)
        return dict(tree)
    for d in lst:
        tree[d[keys[0]]].append(d)
    for k, v in tree.items():
        tree[k] = construct_tree(v, keys[1:])
    return dict(tree)


included_session_context_dict_tree = [ctxt.to_dict() for ctxt in included_session_contexts]

included_session_context_dict_tree

keys = ['format_name', 'animal', 'exper_name', 'session_name']
dict_list = included_session_context_dict_tree.copy()
# ... (rest of your data)]
tree = construct_tree(dict_list, keys)
tree

# Raw ipywidgets version using accordion

In [ ]:
import ipywidgets as widgets
from IPython.display import display

def create_tree_widget(tree, level=0):
    if isinstance(tree, dict):
        acc = widgets.Accordion(children=[])
        for key, value in tree.items():
            child_widget = create_tree_widget(value, level+1)
            acc.children += (child_widget,)
            acc.set_title(len(acc.children)-1, str(key))
        return acc
    elif isinstance(tree, list):
        return widgets.VBox([widgets.Label(str(item)) for item in tree])
    else:
        return widgets.Label(str(tree))

# Your tree variable
# tree = ...

# Create widget tree from data
tree_widget = create_tree_widget(tree)

# Display the widget tree
display(tree_widget)

In [ ]:
selected_leaf = None

def on_leaf_click(b):
    global selected_leaf
    if selected_leaf:
        selected_leaf.button_style = ''
    b.button_style = 'success'
    selected_leaf = b

def create_tree_widget(tree, level=0):
    global selected_leaf
    if isinstance(tree, dict):
        acc = widgets.Accordion(children=[])
        acc.selected_index = list(range(len(tree)))  # Expand all
        for key, value in tree.items():
            child_widget = create_tree_widget(value, level+1)
            acc.children += (child_widget,)
            acc.set_title(len(acc.children)-1, str(key))
        return acc
    elif isinstance(tree, list):
        buttons = []
        for item in tree:
            button = widgets.Button(description=str(item['session_name']), layout=widgets.Layout(width='auto'))
            button.on_click(on_leaf_click)
            buttons.append(button)
        return widgets.VBox(buttons)
    else:
        return widgets.Label(str(tree))
    

# Create widget tree from data
tree_widget = create_tree_widget(tree)

# Display the widget tree
display(tree_widget)

In [ ]:
import ipywidgets as widgets
from IPython.display import display

selected_leaf = None

def toggle_accordion(b):
    acc = b.accordion_reference
    if acc.selected_index is None:
        acc.selected_index = list(range(len(acc.children)))
    else:
        acc.selected_index = None

def on_leaf_click(b):
    global selected_leaf
    if selected_leaf:
        selected_leaf.button_style = ''
    b.button_style = 'success'
    selected_leaf = b

def create_tree_widget(tree, level=0):
    global selected_leaf
    if isinstance(tree, dict):
        acc = widgets.Accordion(children=[])
        toggle_button = widgets.Button(description="Toggle All")
        toggle_button.accordion_reference = acc
        toggle_button.on_click(toggle_accordion)
        
        for key, value in tree.items():
            child_widget = create_tree_widget(value, level+1)
            acc.children += (child_widget,)
            acc.set_title(len(acc.children)-1, str(key))
        return widgets.VBox([toggle_button, acc])
    elif isinstance(tree, list):
        buttons = []
        for item in tree:
            button = widgets.Button(description=str(item['session_name']), layout=widgets.Layout(width='auto'))
            button.on_click(on_leaf_click)
            buttons.append(button)
        return widgets.VBox(buttons)
    else:
        return widgets.Label(str(tree))

# Your tree variable
# tree = ...

# Create widget tree from data
tree_widget = create_tree_widget(tree)

# Display the widget tree
display(tree_widget)


# ipytree version:

In [ ]:


# def on_node_selected(change):
#     if change['new']:  # Node selected
#         print(f"Selected node: {change['owner'].name}")

# def build_tree(node, value, max_depth=20, depth=0):
#     """ 
#         import ipytree as ipyt
#         from IPython.display import display
#         from pyphocorehelpers.gui.Jupyter.TreeWidget import build_tree


#         # Assume curr_computations_results is the root of the data structure you want to explore
#         root_data = a_sess_config.preprocessing_parameters
#         root_node = ipyt.Node('sess.config')
#         build_tree(root_node, root_data)

#         tree = ipyt.Tree()
#         tree.add_node(root_node)
#         display(tree)
#     """
#     if depth > max_depth:
#         return

#     if isinstance(value, dict):
#         for key, child_value in value.items():
#             child_node = ipyt.Node(key)
#             node.add_node(child_node)
#             build_tree(child_node, child_value, max_depth=max_depth, depth=depth+1)
#     else:
#         ## add leaf node:
#         # node.add_node(ipyt.Node(str(value)))
#         leaf_node = ipyt.Node(str(value))
#         leaf_node.observe(on_node_selected, 'selected')
#         node.add_node(leaf_node)



# Assume curr_computations_results is the root of the data structure you want to explore
# root_data = a_sess_config.preprocessing_parameters
# root_node = ipyt.Node('sess.config')

root_data = tree
root_node = ipyt.Node('tree')
build_tree(root_node, root_data)

tree = ipyt.Tree()
tree.add_node(root_node)
display(tree)


In [ ]:
curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15') # DONE. Very good. Many good Pfs, many good replays.


# [*] - indicates bad or session with a problem
# 0, 1, 2, 3, 4, 5, 6, 7, [8], [9], 10, 11, [12], 13, 14, [15], [16], 17, 
# curr_context: IdentifyingContext = good_contexts_list[1] # select the session from all of the good sessions here.
curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15') # DONE. Very good. Many good Pfs, many good replays.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43') # DONE, might be the BEST SESSION, good example session with lots of place cells, clean replays, and clear bar graphs.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31') # DONE, Good Pfs but no good replays
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6') # BAD, 2023-07-14, unsure why still.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19') # DONE, GREAT, both good Pfs and replays!
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25') # DONE, Added replay selections. Very "jumpy" between the starts and ends of the track.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40') # DONE, Added replay selections. A TON of putative replays in general, most bad, but some good.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46') # DONE, added replay selections. Very few (like 12) replays each.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-13_15-22-3') # DONE, Good Pfs, no good epochs.
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30') # DONE, okay replays (selected)
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50') # DONE, very few replays (selected)
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_13-34-40') # BAD
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54') # DONE, one replay each (selected)
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3') # BAD, Good Pfs strangely despite horrible map, no good epochs
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-12_15-25-59') # BAD, No Epochs
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_18-47-52')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-17_12-52-15')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_13-20-55')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_12-38-13')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44') # DONE, good. Many good pfs, many good replays. Noticed very strange jumping off the track in the 3D behavior/spikes viewer. Is there something wrong with this session?
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0') # DONE, good?, replays selected, few
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25') # DONE, very few replays
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_12-15-3')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_22-4-5')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54') # DONE, replays selected, quite a few replays but few are very good.

basedir: Path = output_session_basedir_dict[curr_context]
print(f'basedir: {str(basedir)}')

In [ ]:

# # Read if possible:
# saving_mode = PipelineSavingScheme.SKIP_SAVING
# force_reload = False

# # # # # # Force write:
# # saving_mode = PipelineSavingScheme.TEMP_THEN_OVERWRITE
# # saving_mode = PipelineSavingScheme.OVERWRITE_IN_PLACE
# # force_reload = True

## TODO: if loading is not possible, we need to change the `saving_mode` so that the new results are properly saved.




# 2023-09-21 - `dill` Checking

In [ ]:
# dill.errors(obj, depth=0, exact=False, safe=False) # get errors for objects that fail to pickle
# dill.pickles(obj, exact=False, safe=False, **kwds) # Quick check if object pickles with dill.
# dill.get_objgraph

def dump_session(file_path):
  """Pickle the current python session to be used in the worker.

  Note: Due to the inconsistency in the first dump of dill dump_session we
  create and load the dump twice to have consistent results in the worker and
  the running session. Check: https://github.com/uqfoundation/dill/issues/195
  """
  dill.dump_session(file_path)
  dill.load_session(file_path)
  return dill.dump_session(file_path)

test_session_file = Path('../output/test_pickled_session_2023-09-21.pkl').resolve()
print(f'saving session to {test_session_file}')
dump_session(test_session_file)

# dill.detect.trace(True)
# dill.dump_session(test_session_file) # PicklingError: Can't pickle : it's not the same object as builtins.input
# PicklingError: Can't pickle .NewCol'>: it's not found as tables.description.Col._subclass_from_prefix..NewCol

# with detect.trace('output/dill_trace_2023-09-21.log', mode='w') as log:
#     # log("> D = %r", D)
#     dill.dump_session(test_session_file)


In [ ]:

dill.detect.trace(False)
# dump_session(test_session_file)


In [ ]:
 
dill.dump_session()